In [1]:
import pandas as pd

## Function definitions

In [2]:
def group_by(data, group_column = 'user_id', target_column = 'user_journey', sessions = 'all', count_from = 'last'):
    """
Groups data by the specified group_column and concatenates target_column values for each group. Other columns are set to their first record in the group. Allows choosing whether to aggregate all records or only the first/last sessions.

Parameters:
data (pandas.DataFrame): Data to be grouped
group_column (str): Column to group by (default: 'user_id')
target_column (str): Column to concatenate (default: 'user_journey')
sessions (str or int): Number of records to aggregate ('all', 'all_except_last', or int)
count_from (str): 'first' or 'last', indicates where to start aggregation (default: 'last')

Returns:
df (pandas.DataFrame): Grouped DataFrame (original data unchanged)
""" 
    # Create a new dataframe to populate with the results
    df = pd.DataFrame(columns = data.columns)
    
    
    # Set the start and end index for the sessions that should be aggregated
    if sessions == "all":
        start = 0
        end = None
    
    elif sessions == "all_except_last":
        start = 0
        end = -1
    
    elif count_from == "last":
        start = - sessions
        end = None
    
    elif count_from == "first":
        start = 0
        end = sessions
    
    
    # Obtain the set of unique user_id values
    groups = set(data[group_column])
    
    # Iterate over each group (unordered, as sets are unordered)
    for group_value in groups:
        
        group_mask = list(data[group_column] == group_value) # create a mask to filter only the rows with group_value
        group_table = data[group_mask] # a view of the data corresponding to a single group
        
        # Obtain an aggregated user journey string corresponding to the group
        user_journey = "-".join(list(group_table[target_column])[start:end])
        
        # Append a row to the new DataFrame corresponding to a single (aggregated) group
        new_index = len(df)
        df.loc[new_index] = group_table.iloc[0].copy() # all values except target_column are copied from the first occurence
        df.loc[new_index][target_column] = user_journey

    
    # Reset the index (precautionary) and sort the new DataFrame by user_id
    df.sort_values(by=[group_column], ignore_index = True, inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    
    
    return df
    

In [9]:
def remove_pages(data, pages = [], target_column = 'user_journey'):
    """
Returns a new DataFrame where the specified pages are removed from the user journey strings.

Parameters:
data (pandas.DataFrame): The DataFrame containing the user journeys data
pages (list or set): List or set of strings representing pages to remove (default: [])
target_column (str): The column containing user journey strings (default: 'user_journey')

Returns:
df (pandas.DataFrame): New DataFrame with updated data (original data unchanged)
"""


    # Make a copy of the dataframe as to not accidentaly modify the original data
    df = data.copy()
    pages = set(pages)
    
    if len(pages) == 0:
        return df
    
    # Obtain a list of all user journey strings -> ["page1-page2-...pageN", ...]
    user_journey = list(df[target_column])
    
    # Split the journey strings into pages -> [["page1", "page2", ..."pageN"], ...]
    user_journey = [journey.split('-') for journey in user_journey]
    
    # Include only pages that are not contained in the set of pages to remove
    user_journey = [[page for page in journey if page not in pages] for journey in user_journey]
    
    # Combine the pages into strings again -> ["page1-page2-...pageN", ...]
    user_journey = ["-".join(i) for i in user_journey]
    
    
    # Update the user journey column with the new strings
    df[target_column] = user_journey
    
    
    
    return df
    

In [3]:
def remove_page_duplicates(data, target_column = 'user_journey'):
    """
Returns a new DataFrame where consecutive page duplicates are removed from the user journey strings.
Non-consecutive duplicates are not removed.

Parameters:
data (pandas.DataFrame): The DataFrame containing the user journeys data
target_column (str): The column containing user journey strings (default: 'user_journey')

Returns:
df (pandas.DataFrame): New DataFrame with updated data (original data unchanged)
"""

    
    # Make a copy of the dataframe as to not accidentaly modify the original data
    df = data.copy()
    
    
    # Obtain a list of all user journey strings -> ["page1-page2-...pageN", ...]
    user_journey = list(df[target_column])
    
    # Split the journey strings into pages -> [["page1", "page2", ..."pageN"], ...]
    user_journey = [journey.split('-') for journey in user_journey]
    
    
    # Loop through all pages and remove consecutive duplicates
    for i in range(len(user_journey)):
        j = 1
        
        while(j < len(user_journey[i])):
            
            if user_journey[i][j-1] == user_journey[i][j]: # Check and remove the page if the next one is the same
                user_journey[i].pop(j)
            
            else:
                j += 1 # Move to the next page
    
    
    # Combine the pages into strings again -> ["page1-page2-...pageN", ...]
    user_journey = ["-".join(i) for i in user_journey]
    
    
    # Update the user journey column with the new strings
    df[target_column] = user_journey
    
    
    
    return df
    

## Main

In [5]:
# Load the unprocessed user journey data (SQL query result)
raw_data = pd.read_csv('/Users/premann/Downloads/user_journey_raw.csv')

In [6]:
# Check how the data looks
raw_data.head()

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,2980248,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,2992252,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,3709807,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...


In [7]:
# Create a copy of the raw data that will be updated and drop the "session_id" column as we don't need it
clean_data = raw_data.copy().drop("session_id", axis=1)

In [10]:
# Preprocessing of the data
clean_data = group_by(clean_data)
clean_data = remove_pages(clean_data, [])
clean_data = remove_page_duplicates(clean_data)

In [11]:
clean_data.head()

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Other-Sign up-Log in-Homepage-...
1,3395,Annual,Other-Pricing-Sign up-Log in-Homepage-Pricing-...
2,10107,Annual,Homepage-Career tracks-Homepage-Career tracks-...
3,11145,Monthly,Homepage-Log in-Homepage-Log in-Homepage-Log i...
4,12400,Monthly,Homepage-Career tracks-Sign up-Log in-Other-Ca...


In [12]:
# Save the preprocessed data
clean_data.to_csv('/Users/premann/Downloads/user_journey.csv', index = False)